# Example: Custom `MoveStrategy`: RepEx-Shoot-Repex

One of the powerful features in OpenPathSampling is that it is very easy to develop new Monte Carlo movers for path space. This example shows how easy it is to try out a new type of move. The particular move we use here can be easily described as a simple combination of existing moves, so we don't even need to define a new `PathMover` subclass. We just define a custom `MoveStrategy` that creates the desired `PathMover`, and use that directly.

The idea implemented here is pretty simple. Our standard path movers treat shooting and replica exchange separately, and each move is a single shooting (one ensemble) or a single replica exchange (swap one pair). But maybe you could get better replica exchange behavior by trying all the replica exchange moves, and then trying all the shooting moves. Note that, to satisfy detailed balance, you really have to do all the replica exchange moves, then all the shooting moves, then all the replica exchange moves in the reverse order from before. To measure how this affects travel in replica space, we'll use the replica round trip time (normalized to the total number of shooting moves per ensemble).

In [1]:
import openpathsampling as paths
import numpy as np

# Set up the simulation

## Set up the engine

In [2]:
import openpathsampling.engines.toy as toys
pes = (toys.OuterWalls([1.0,1.0], [0.0,0.0]) + 
       toys.Gaussian(-0.7, [12.0, 0.5], [-0.5, 0.0]) +
       toys.Gaussian(-0.7, [12.0, 0.5], [0.5, 0.0]))

topology = toys.Topology(n_spatial=2, masses=[1.0, 1.0], pes=pes)

engine = toys.Engine(options={'integ': toys.LangevinBAOABIntegrator(dt=0.02, temperature=0.1, gamma=2.5),
                              'n_frames_max': 5000,
                              'n_steps_per_frame': 10},
                     topology=topology)

template = toys.Snapshot(coordinates=np.array([[0.0, 0.0]]),
                         velocities=np.array([[0.0, 0.0]]),
                         engine=engine)

## Set up CV and volumes (states, interfaces)

In [3]:
# states are volumes in a CV space: define the CV
def xval(snapshot):
    return snapshot.xyz[0][0]

cv = paths.FunctionCV("xval", xval)

stateA = paths.CVDefinedVolume(cv, float("-inf"), -0.5).named("A")
stateB = paths.CVDefinedVolume(cv, 0.5, float("inf")).named("B")
interfaces_AB = paths.VolumeInterfaceSet(cv, float("-inf"), [-0.5, -0.4, -0.3, -0.2, -0.1, 0.0])

## Set up network

In [4]:
network = paths.MISTISNetwork([(stateA, interfaces_AB, stateB)])

## Define a custom strategy

This is the main point of this example: Here we create a custom `MoveStrategy`, which includes the creation of the custom mover. Note that the custom mover itself is quite simple. It takes a bunch of moves that have already been defined, and combines them into a different move.

This is a `GROUP`-level mover, meaning that it only acts after you've already movers in the `SIGNATURE` level. Because of this, all it has to do is to reorganize the movers that already exist.

In [5]:
import openpathsampling.analysis.move_strategy as strategies # TODO: handle this better
# example: custom subclass of `MoveStrategy`
class RepExShootRepExStrategy(strategies.MoveStrategy):
    _level = strategies.levels.GROUP
    # we define an init function mainly to set defaults for `replace` and `group`
    def __init__(self, ensembles=None, group="repex_shoot_repex", replace=True, network=None):
        super(RepExShootRepExStrategy, self).__init__(
            ensembles=ensembles, group=group, replace=replace
        )
            
    def make_movers(self, scheme):
        # if we replace, we remove these groups from the scheme.movers dictionary
        if self.replace:
            repex_movers = scheme.movers.pop('repex')
            shoot_movers = scheme.movers.pop('shooting')
        else:
            repex_movers = scheme.movers['repex']
            shoot_movers = scheme.movers['shooting']
        # combine into a list for the SequentialMover
        mover_list = repex_movers + shoot_movers + list(reversed(repex_movers))
        combo_mover = paths.SequentialMover(mover_list)
        return [combo_mover]

## Create two move schemes: Default and Custom

In [6]:
default_scheme = paths.DefaultScheme(network, engine)

In [8]:
custom_scheme = paths.DefaultScheme(network, engine)
custom_scheme.append(RepExShootRepExStrategy())

# Get initial conditions

In [12]:
initial_samples = paths.FullBootstrapping(transition=network.sampling_transitions[0],
                                          snapshot=template,
                                          engine=engine).run()

DONE! Completed Bootstrapping cycle step 132 in ensemble 6/6.


In [13]:
transition = network.sampling_transitions[0]
minus_sample = network.minus_ensembles[0].populate_minus_ensemble(
    partial_traj=initial_samples[transition.ensembles[0]].trajectory,
    minus_replica_id=-1,
    engine=engine
)
initial_samples = initial_samples.apply_samples(minus_sample)

In [14]:
initial_samples.sanity_check()

In [15]:
print "Default Scheme:", default_scheme.initial_conditions_report(initial_samples)
print "Custom Scheme:", custom_scheme.initial_conditions_report(initial_samples)

Default Scheme: No missing ensembles.
No extra ensembles.

Custom Scheme: No missing ensembles.
No extra ensembles.



# Run each of the simulations

In [16]:
n_tries_per_shooting = 100

In [17]:
# take the number of steps from a single ensemble shooting
n_steps = default_scheme.n_steps_for_trials(
    mover=default_scheme.movers['shooting'][0],
    n_attempts=n_tries_per_shooting
)
n_steps = int(n_steps)+1
print n_steps

1171


In [18]:
default_storage = paths.Storage("default_scheme.nc", "w")

In [19]:
default_calc = paths.PathSampling(
    storage=default_storage,
    sample_set=initial_samples,
    move_scheme=default_scheme
)

In [20]:
default_calc.run(n_steps)

Working on Monte Carlo cycle number 1171.
DONE! Completed 1171 Monte Carlo cycles.


In [21]:
# in repex_shoot_repex, one move shoots all the ensembles
n_steps = custom_scheme.n_steps_for_trials(
    mover=custom_scheme.movers['repex_shoot_repex'],
    n_attempts=n_tries_per_shooting
)
n_steps = int(n_steps)+1
print n_steps

421


In [22]:
custom_storage = paths.Storage("custom_scheme.nc", "w")

In [23]:
custom_calc = paths.PathSampling(
    storage=custom_storage,
    sample_set=initial_samples,
    move_scheme=custom_scheme
)

In [24]:
custom_calc.run(n_steps)

Working on Monte Carlo cycle number 421.
DONE! Completed 421 Monte Carlo cycles.


# Analyze the results

## A few checks that we are making a fair comparison

### The scheme should be as expected

In [25]:
default_scheme.move_summary(default_storage.steps)

repex ran 21.435% (expected 21.37%) of the cycles with acceptance 78/251 (31.08%)
shooting ran 49.445% (expected 51.28%) of the cycles with acceptance 390/579 (67.36%)
minus ran 1.708% (expected 1.71%) of the cycles with acceptance 20/20 (100.00%)
pathreversal ran 27.412% (expected 25.64%) of the cycles with acceptance 294/321 (91.59%)


In [26]:
custom_scheme.move_summary(custom_storage.steps)

repex_shoot_repex ran 26.128% (expected 23.81%) of the cycles with acceptance 110/110 (100.00%)
minus ran 4.751% (expected 4.76%) of the cycles with acceptance 20/20 (100.00%)
pathreversal ran 69.121% (expected 71.43%) of the cycles with acceptance 207/291 (71.13%)


### The number of snapshots generated by each should be similar

In [27]:
print len(default_storage.snapshots), len(custom_storage.snapshots)

52110 61540


### Check that we have about the same number of shooting moves per ensemble for each scheme

In [28]:
default_scheme.move_summary(default_storage.steps, "shooting")

OneWayShootingMover [TISTransition] 0 ran 7.942% (expected 8.55%) of the cycles with acceptance 75/93 (80.65%)
OneWayShootingMover [TISTransition] 3 ran 8.369% (expected 8.55%) of the cycles with acceptance 54/98 (55.10%)
OneWayShootingMover [TISTransition] 2 ran 6.832% (expected 8.55%) of the cycles with acceptance 59/80 (73.75%)
OneWayShootingMover [TISTransition] 5 ran 7.942% (expected 8.55%) of the cycles with acceptance 49/93 (52.69%)
OneWayShootingMover [TISTransition] 1 ran 9.735% (expected 8.55%) of the cycles with acceptance 85/114 (74.56%)
OneWayShootingMover [TISTransition] 4 ran 8.625% (expected 8.55%) of the cycles with acceptance 68/101 (67.33%)


In [29]:
custom_scheme.move_summary(custom_storage.steps, "repex_shoot_repex")

Sequential ran 26.128% (expected 23.81%) of the cycles with acceptance 110/110 (100.00%)


## Analyze the output to compare the efficiency

### Count the number of round trips done

In [38]:
default_repx_net = paths.ReplicaNetwork(default_scheme, default_storage.steps)

In [39]:
default_trips = default_repx_net.trips(bottom=network.minus_ensembles[0], top=network.all_ensembles[-1])

In [40]:
n_default_round_trips = len(default_trips['round'])
print n_default_round_trips

7


In [41]:
custom_repx_net = paths.ReplicaNetwork(custom_scheme, custom_storage.steps)

In [42]:
custom_trips = custom_repx_net.trips(bottom=network.minus_ensembles[0], top=network.all_ensembles[-1])

In [43]:
n_custom_round_trips = len(custom_trips['round'])
print n_custom_round_trips

8


### Check the replica flow for each scheme

In [44]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
# TODO: move some of the pandas stuff into the analysis objects

In [45]:
ensemble_list = network.minus_ensembles + network.all_ensembles
ensemble_id = {e: ensemble_list.index(e) for e in ensemble_list}
default_flow = pd.Series(default_repx_net.flow(bottom=network.minus_ensembles[0], top=network.all_ensembles[-1]))
default_flow.index = [ensemble_id[e] for e in default_flow.index]
# TODO: this doesn't seem to be right

In [47]:
default_flow

0    0.0
1    0.0
2    0.0
3    0.0
4    0.0
5    0.0
6    0.0
dtype: float64